In [2]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
# os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
# sys.path.insert(0,nb_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install --upgrade --target=$nb_path torch
# !pip install --upgrade --target=$nb_path torch-geometric
# !pip install --upgrade --target=$nb_path torch-sparse
# !pip install --upgrade --target=$nb_path torch-scatter
# !pip install --upgrade --target=$nb_path torch-cluster 
# !pip install --upgrade --target=$nb_path pytorch_lightning
! pip install --quiet "torch-scatter" "ipython[notebook]" "torchmetrics>=0.7" "torch>=1.8" "torch-cluster" "torch-spline-conv" "setuptools==59.5.0" "torch-sparse<0.6.13" "torch-geometric==2.0.2" "pytorch-lightning>=1.4"

In [ ]:
import torch
import warnings
warnings.filterwarnings('ignore')

import os

import pandas as pd
import numpy as np
import seaborn as sns
import math
import json
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mc
from matplotlib import cm
import seaborn as sns
import colorsys
from tqdm import tqdm

from sklearn.model_selection import train_test_split, KFold


import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.utils.convert import to_networkx
from torch_geometric.nn import GCNConv
import torch_geometric.data as geom_data
import torch_geometric.nn as geom_nn
import torch_geometric

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
class SocialGNN(torch.nn.Module):
    def __init__(self,num_of_feat,f):
        super(SocialGNN, self).__init__()
        self.conv1 = GCNConv(num_of_feat, f)
        self.conv2 = GCNConv(f, 2)

    def forward(self, data):
        x = data.x.float()
        edge_index =  data.edge_index
        x = self.conv1(x=x, edge_index=edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x


In [6]:
df = pd.read_excel("GNN_data_f.xlsx", index_col=False)
df = df.drop(df.columns[[0]], axis=1)  
df

,Chr_x,Gene symbol,loc,z-score,int_size,eqtl_genes,DISEASE/TRAIT,l_break,r_break,seq
0,chr19,-,47774772,1.641685e+07,NaN,"['ENSG00000105321', 'ENSG00000197405', 'ENSG00...",NaN,47774272,47775272,AGGCAGGAAGCCCCTTCCTCCTGTCTACTCTCCACCCAGGCATCAC...
1,chr18,SMAD4,48609968,4.044151e+06,NaN,"['ENSG00000141646', 'ENSG00000082212', 'ENSG00...",NaN,48609468,48610468,TGAAAGGTTTCATTGCTTCCACTTGAATGCTGCTCTTACAAAAACT...
2,chr18,SMAD4,48610474,4.044151e+06,NaN,NaN,NaN,48609974,48610974,AAAGTAATGGCTCTGGGTTGGGCCAGACAGTTGCACTCTCTAGTTT...
3,chr1,-,212459056,1.049522e+06,412.0,NaN,NaN,212458556,212459556,CTCACGCCGTCGCTGCCGCTTAGCCGTGTGATTGGCTAAGAGGGCT...
4,chr15,NR2F2,96873540,9.413342e+05,NaN,NaN,NaN,96873040,96874040,GAAAATTGATTCTAATTGCATCATGACAATCAGATGAAGTCTATTT...
...,...,...,...,...,...,...,...,...,...,...
7785,chr12,BCAT1,24965636,9.017178e+00,NaN,NaN,NaN,24965136,24966136,TGAGGAAACTGCAATGTGTTATTGCACATTTTAGTTAAAAACTAAT...
7786,chr4,-,83345632,8.958757e+00,NaN,['ENSG00000145293'],NaN,83345132,83346132,AAACATTTTGTTCAATCTTGTGGCGGCTCTGAGAAAACACAACTAA...
7787,chr12,-,27182338,8.381627e+00,NaN,"['ENSG00000111790', 'ENSG00000152944', 'ENSG00...",NaN,27181838,27182838,TCTGAGCTTTGAGCAAGAAAGTTTTGGAAATTGTGTTGCTTTTAAG...
7788,chr12,VEZT,95696030,8.337806e+00,NaN,NaN,NaN,95695530,95696530,TCTATGTTGCTGTCCTAGAAAGGAAATTGCATGATGAATCTAGATT...


In [7]:
df["target"] = ((df["z-score"] > 200) & (~df["eqtl_genes"].isna()) & (~df["DISEASE/TRAIT"].isna())) * 1
df["target"].sum()

78

In [8]:
df

,Chr_x,Gene symbol,loc,z-score,int_size,eqtl_genes,DISEASE/TRAIT,l_break,r_break,seq,target
0,chr19,-,47774772,1.641685e+07,NaN,"['ENSG00000105321', 'ENSG00000197405', 'ENSG00...",NaN,47774272,47775272,AGGCAGGAAGCCCCTTCCTCCTGTCTACTCTCCACCCAGGCATCAC...,0
1,chr18,SMAD4,48609968,4.044151e+06,NaN,"['ENSG00000141646', 'ENSG00000082212', 'ENSG00...",NaN,48609468,48610468,TGAAAGGTTTCATTGCTTCCACTTGAATGCTGCTCTTACAAAAACT...,0
2,chr18,SMAD4,48610474,4.044151e+06,NaN,NaN,NaN,48609974,48610974,AAAGTAATGGCTCTGGGTTGGGCCAGACAGTTGCACTCTCTAGTTT...,0
3,chr1,-,212459056,1.049522e+06,412.0,NaN,NaN,212458556,212459556,CTCACGCCGTCGCTGCCGCTTAGCCGTGTGATTGGCTAAGAGGGCT...,0
4,chr15,NR2F2,96873540,9.413342e+05,NaN,NaN,NaN,96873040,96874040,GAAAATTGATTCTAATTGCATCATGACAATCAGATGAAGTCTATTT...,0
...,...,...,...,...,...,...,...,...,...,...,...
7785,chr12,BCAT1,24965636,9.017178e+00,NaN,NaN,NaN,24965136,24966136,TGAGGAAACTGCAATGTGTTATTGCACATTTTAGTTAAAAACTAAT...,0
7786,chr4,-,83345632,8.958757e+00,NaN,['ENSG00000145293'],NaN,83345132,83346132,AAACATTTTGTTCAATCTTGTGGCGGCTCTGAGAAAACACAACTAA...,0
7787,chr12,-,27182338,8.381627e+00,NaN,"['ENSG00000111790', 'ENSG00000152944', 'ENSG00...",NaN,27181838,27182838,TCTGAGCTTTGAGCAAGAAAGTTTTGGAAATTGTGTTGCTTTTAAG...,0
7788,chr12,VEZT,95696030,8.337806e+00,NaN,NaN,NaN,95695530,95696530,TCTATGTTGCTGTCCTAGAAAGGAAATTGCATGATGAATCTAGATT...,0


In [9]:
df_new = pd.DataFrame()
df_new['id'] = df['Chr_x'] + '_' + df['loc'].apply(str)
df_new['data'] = df['seq']
df_new['target'] = df['target']
df_new

,id,data,target
0,chr19_47774772,AGGCAGGAAGCCCCTTCCTCCTGTCTACTCTCCACCCAGGCATCAC...,0
1,chr18_48609968,TGAAAGGTTTCATTGCTTCCACTTGAATGCTGCTCTTACAAAAACT...,0
2,chr18_48610474,AAAGTAATGGCTCTGGGTTGGGCCAGACAGTTGCACTCTCTAGTTT...,0
3,chr1_212459056,CTCACGCCGTCGCTGCCGCTTAGCCGTGTGATTGGCTAAGAGGGCT...,0
4,chr15_96873540,GAAAATTGATTCTAATTGCATCATGACAATCAGATGAAGTCTATTT...,0
...,...,...,...
7785,chr12_24965636,TGAGGAAACTGCAATGTGTTATTGCACATTTTAGTTAAAAACTAAT...,0
7786,chr4_83345632,AAACATTTTGTTCAATCTTGTGGCGGCTCTGAGAAAACACAACTAA...,0
7787,chr12_27182338,TCTGAGCTTTGAGCAAGAAAGTTTTGGAAATTGTGTTGCTTTTAAG...,0
7788,chr12_95696030,TCTATGTTGCTGTCCTAGAAAGGAAATTGCATGATGAATCTAGATT...,0


In [10]:
df_new[[str(i) for i in range(1001)]] = df_new['data'].apply(lambda x: ' '.join(x)).str.split(' ', 1001, expand=True)
x = df_new['data'].apply(lambda x: ' '.join(x)).str.split(' ', 1001, expand=True)
df_new

,id,data,target,0,1,2,3,4,5,6,...,991,992,993,994,995,996,997,998,999,1000
0,chr19_47774772,AGGCAGGAAGCCCCTTCCTCCTGTCTACTCTCCACCCAGGCATCAC...,0,A,G,G,C,A,G,G,...,C,T,C,T,C,T,T,G,G,G
1,chr18_48609968,TGAAAGGTTTCATTGCTTCCACTTGAATGCTGCTCTTACAAAAACT...,0,T,G,A,A,A,G,G,...,A,T,T,G,C,G,T,G,C,A
2,chr18_48610474,AAAGTAATGGCTCTGGGTTGGGCCAGACAGTTGCACTCTCTAGTTT...,0,A,A,A,G,T,A,A,...,T,T,G,T,C,A,T,T,G,A
3,chr1_212459056,CTCACGCCGTCGCTGCCGCTTAGCCGTGTGATTGGCTAAGAGGGCT...,0,C,T,C,A,C,G,C,...,C,G,C,A,G,A,G,G,C,A
4,chr15_96873540,GAAAATTGATTCTAATTGCATCATGACAATCAGATGAAGTCTATTT...,0,G,A,A,A,A,T,T,...,C,G,C,G,G,T,G,T,G,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7785,chr12_24965636,TGAGGAAACTGCAATGTGTTATTGCACATTTTAGTTAAAAACTAAT...,0,T,G,A,G,G,A,A,...,T,G,T,T,T,C,A,T,T,A
7786,chr4_83345632,AAACATTTTGTTCAATCTTGTGGCGGCTCTGAGAAAACACAACTAA...,0,A,A,A,C,A,T,T,...,C,A,C,T,T,T,A,T,T,T
7787,chr12_27182338,TCTGAGCTTTGAGCAAGAAAGTTTTGGAAATTGTGTTGCTTTTAAG...,0,T,C,T,G,A,G,C,...,A,C,T,A,A,C,A,G,G,A
7788,chr12_95696030,TCTATGTTGCTGTCCTAGAAAGGAAATTGCATGATGAATCTAGATT...,0,T,C,T,A,T,G,T,...,A,A,T,G,A,G,C,C,A,T


In [11]:
for i in range(1001):
  if len(x.iloc[i].unique()) != 4:
    print(x.iloc[i].unique())
    print(i)

In [12]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit([['A' for i in range(1001)],['C' for i in range(1001)],['T' for i in range(1001)],['G' for i in range(1001)]])
xnew = enc.transform(x).toarray()
xnew

array([[1., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 1., 0.]])

In [13]:
Tensor(np.array(np.array_split(xnew[0], 1001))).size()


torch.Size([1001, 4])

In [14]:
# data = Data(x=x, edge_index=edge_index, ...)

# # Add additional arguments to `data`:
# data.train_idx = torch.tensor([...], dtype=torch.long)
# data.test_mask = torch.tensor([...], dtype=torch.bool)

In [15]:
edge_index = np.zeros(shape=(2, 2000))
st_0 = [0]
for i in range(1, 1000):
  st_0.append(i)
  st_0.append(i)
st_0.append(1000)

st_1 = [1]
for i in range(1, 1000):
  st_1.append(i - 1)
  st_1.append(i + 1)
st_1.append(999)

edge_index[0, :] = st_0
edge_index[1, :] = st_1
edge_index = Tensor(edge_index)
edge_index

tensor([[   0.,    1.,    1.,  ...,  999.,  999., 1000.],
        [   1.,    0.,    2.,  ...,  998., 1000.,  999.]])

In [16]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

data_list = []
for i in range(7790):
   x = Tensor(np.array(np.array_split(xnew[i], 1001)))
   data = Data(x=x, edge_index=edge_index.long(), y=Tensor([df_new['target'][0]]).long())
   data_list.append(data)
loader = DataLoader(data_list, batch_size=64)

In [17]:
data_list[0].batch

In [18]:
gnn_layer_by_name = {"GCN": geom_nn.GCNConv, "GAT": geom_nn.GATConv, "GraphConv": geom_nn.GraphConv}

In [19]:
class GNNModel(nn.Module):
    def __init__(
        self,
        c_in,
        c_hidden,
        c_out,
        num_layers=2,
        layer_name="GCN",
        dp_rate=0.1,
        **kwargs,
    ):
        """
        Args:
            c_in: Dimension of input features
            c_hidden: Dimension of hidden features
            c_out: Dimension of the output features. Usually number of classes in classification
            num_layers: Number of "hidden" graph layers
            layer_name: String of the graph layer to use
            dp_rate: Dropout rate to apply throughout the network
            kwargs: Additional arguments for the graph layer (e.g. number of heads for GAT)
        """
        super().__init__()
        gnn_layer = gnn_layer_by_name[layer_name]

        layers = []
        in_channels, out_channels = c_in, c_hidden
        for l_idx in range(num_layers - 1):
            layers += [
                gnn_layer(in_channels=in_channels, out_channels=out_channels, **kwargs),
                nn.ReLU(inplace=True),
                nn.Dropout(dp_rate),
            ]
            in_channels = c_hidden
        layers += [gnn_layer(in_channels=in_channels, out_channels=c_out, **kwargs)]
        self.layers = nn.ModuleList(layers)

    def forward(self, x, edge_index):
        """
        Args:
            x: Input features per node
            edge_index: List of vertex index pairs representing the edges in the graph (PyTorch geometric notation)
        """
        for layer in self.layers:
            # For graph layers, we need to add the "edge_index" tensor as additional input
            # All PyTorch Geometric graph layer inherit the class "MessagePassing", hence
            # we can simply check the class type.
            if isinstance(layer, geom_nn.MessagePassing):
                x = layer(x, edge_index)
            else:
                x = layer(x)
        return x

In [20]:
class GraphGNNModel(nn.Module):
    def __init__(self, c_in, c_hidden, c_out, dp_rate_linear=0.5, **kwargs):
        """
        Args:
            c_in: Dimension of input features
            c_hidden: Dimension of hidden features
            c_out: Dimension of output features (usually number of classes)
            dp_rate_linear: Dropout rate before the linear layer (usually much higher than inside the GNN)
            kwargs: Additional arguments for the GNNModel object
        """
        super().__init__()
        self.GNN = GNNModel(c_in=c_in, c_hidden=c_hidden, c_out=c_hidden, **kwargs)  # Not our prediction output yet!
        self.head = nn.Sequential(nn.Dropout(dp_rate_linear), nn.Linear(c_hidden, c_out))

    def forward(self, x, edge_index, batch_idx):
        """
        Args:
            x: Input features per node
            edge_index: List of vertex index pairs representing the edges in the graph (PyTorch geometric notation)
            batch_idx: Index of batch element for each node
        """
        x = self.GNN(x, edge_index)
        x = geom_nn.global_mean_pool(x, batch_idx)  # Average pooling
        x = self.head(x)
        return x

In [21]:
class GraphLevelGNN(pl.LightningModule):
    def __init__(self, **model_kwargs):
        super().__init__()
        # Saving hyperparameters
        self.save_hyperparameters()

        self.model = GraphGNNModel(**model_kwargs)
        self.loss_module = nn.BCEWithLogitsLoss() if self.hparams.c_out == 1 else nn.CrossEntropyLoss()

    def forward(self, data, mode="train"):
        x, edge_index, batch_idx = data.x, data.edge_index, data.batch
        x = self.model(x, edge_index, batch_idx)
        x = x.squeeze(dim=-1)

        if self.hparams.c_out == 1:
            preds = (x > 0).float()
            data.y = data.y.float()
        else:
            preds = x.argmax(dim=-1)
        loss = self.loss_module(x, data.y)
        acc = (preds == data.y).sum().float() / preds.shape[0]
        return loss, acc

    def configure_optimizers(self):
        # High lr because of small dataset and small model
        optimizer = optim.AdamW(self.parameters(), lr=1e-2, weight_decay=0.0)
        return optimizer

    def training_step(self, batch, batch_idx):
        loss, acc = self.forward(batch, mode="train")
        self.log("train_loss", loss)
        self.log("train_acc", acc)
        return loss

    def validation_step(self, batch, batch_idx):
        _, acc = self.forward(batch, mode="val")
        self.log("val_acc", acc)

    def test_step(self, batch, batch_idx):
        _, acc = self.forward(batch, mode="test")
        self.log("test_acc", acc)

In [22]:
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 256 if AVAIL_GPUS else 64
# Path to the folder where the datasets are/should be downloaded
DATASET_PATH = os.environ.get("PATH_DATASETS", "data/")
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = os.environ.get("PATH_CHECKPOINT", "saved_models/GNNs/")

In [23]:
from random import shuffle

torch.manual_seed(42)
shuffle(data_list)
train_dataset = data_list[:5000]
test_dataset = data_list[5000:]

In [24]:
graph_train_loader = geom_data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
graph_val_loader = geom_data.DataLoader(test_dataset, batch_size=BATCH_SIZE)  # Additional loader for a larger datasets
graph_test_loader = geom_data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [29]:
tu_dataset = torch_geometric.datasets.TUDataset(root=DATASET_PATH, name="MUTAG")

In [30]:
tu_dataset.num_classes, tu_dataset.num_node_features

(2, 7)

In [31]:
def train_graph_classifier(model_name, **model_kwargs):
    pl.seed_everything(42)

    # Create a PyTorch Lightning trainer with the generation callback
    trainer = pl.Trainer(
        gpus=0,
        max_epochs=100,
        progress_bar_refresh_rate=0,
    )
    trainer.logger._default_hp_metric = None

    pl.seed_everything(42)
    model = GraphLevelGNN(
        c_in=4,
        c_out=1,
        **model_kwargs,
    )
    trainer.fit(model, graph_train_loader, graph_val_loader)

    # Test best model on validation and test set
    train_result = trainer.test(model, dataloaders=graph_train_loader, verbose=False)
    test_result = trainer.test(model, dataloaders=graph_test_loader, verbose=False)
    result = {"test": test_result[0]["test_acc"], "train": train_result[0]["test_acc"]}
    return model, result

In [32]:
import torch.optim as optim

In [33]:
criterion=nn.CrossEntropyLoss()

def masked_loss(predictions,labels,mask):
    mask=mask.float()
    mask=mask/torch.mean(mask)
    loss=criterion(predictions,labels)
    loss=loss*mask
    loss=torch.mean(loss)
    return (loss)    

def masked_accuracy(predictions,labels,mask):
    mask=mask.float()
    mask/=torch.mean(mask)
    accuracy=(torch.argmax(predictions,axis=1)==labels).long()
    accuracy=mask*accuracy
    accuracy=torch.mean(accuracy)
    return (accuracy)  

In [34]:
net = GraphLevelGNN(
        c_in=4,
        c_out=1,
        c_hidden=2, layer_name="GraphConv", num_layers=2, dp_rate_linear=0.5, dp_rate=0.0
)

In [39]:
# out=net(graph_train_loader)
# out

AttributeError: ignored

In [ ]:
def train_social(net,data,epochs=10,lr=0.01):
    optimizer = torch.optim.Adam(net.parameters(), lr=lr) # 00001
    best_accuracy=0.0
    
    train_losses=[]
    train_accuracies=[]

    val_losses=[]
    val_accuracies=[]

    test_losses=[]
    test_accuracies=[]
    
    for ep in range(epochs+1):
        optimizer.zero_grad()
        out=net(data)
        loss=masked_loss(predictions=out,
                         labels=data.y,
                         mask=data.train_mask)
        loss.backward()
        optimizer.step()
        train_losses+=[loss]
        train_accuracy=masked_accuracy(predictions=out,
                                       labels=data.y, 
                                       mask=data.train_mask)
        train_accuracies+=[train_accuracy]
        
        val_loss=masked_loss(predictions=out,
                             labels=data.y, 
                             mask=data.val_mask)
        val_losses+=[val_loss]
        
        val_accuracy=masked_accuracy(predictions=out,
                                     labels=data.y, 
                                     mask=data.val_mask)
        val_accuracies+=[val_accuracy]

        test_accuracy=masked_accuracy(predictions=out,
                                      labels=data.y, 
                                      mask=data.test_mask)
        test_accuracies+=[test_accuracy]
        if np.round(val_accuracy,4)> np.round(best_accuracy ,4):
            print("Epoch {}/{}, Train_Loss: {:.4f}, Train_Accuracy: {:.4f}, Val_Accuracy: {:.4f}, Test_Accuracy: {:.4f}"
                      .format(ep+1,epochs, loss.item(), train_accuracy, val_accuracy,  test_accuracy))
            best_accuracy=val_accuracy

    plt.plot(train_losses)  
    plt.plot(val_losses)
    plt.plot(test_losses)  
    plt.show()
    
    plt.plot(train_accuracies) 
    plt.plot(val_accuracies)
    plt.plot(test_accuracies) 
    plt.show()

In [ ]:
model, result = train_graph_classifier(
    model_name="GraphConv", c_hidden=2, layer_name="GraphConv", num_layers=2, dp_rate_linear=0.5, dp_rate=0.0
)

Global seed set to 42
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=0)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42

  | Name        | Type              | Params
--------------------------------------------------
0 | model       | GraphGNNModel     | 31    
1 | loss_module | BCEWithLogitsLoss | 0     
--------------------------------------------------
31        Tr

In [ ]:
print("Train performance: %4.2f%%" % (100.0 * result["train"]))
print("Test performance:  %4.2f%%" % (100.0 * result["test"]))

In [ ]:
gr = 0
for i in x.iterrows():
  for node ini[1])
  gr += 1

In [ ]:
from torch import Tensor
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='.', name='Cora')

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        # x: Node feature matrix of shape [num_nodes, in_channels]
        # edge_index: Graph connectivity matrix of shape [2, num_edges]
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

model = GCN(dataset.num_features, 16, dataset.num_classes)

Processing...
Done!


In [ ]:
data = dataset[0]
data['x'].size(), data['edge_index'].size()

(torch.Size([2708, 1433]), torch.Size([2, 10556]))

In [ ]:
data['edge_index']

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])

In [ ]:

data = dataset[0]
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(200):
    pred = model(data.x, data.edge_index)
    loss = F.cross_entropy(pred[data.train_mask], data.y[data.train_mask])

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
pred = model(data.x, data.edge_index)
loss = F.cross_entropy(pred[~data.train_mask], data.y[~data.train_mask])
loss

tensor(0.8348, grad_fn=<NllLossBackward0>)

In [ ]:
pred

tensor([[-2.2942, -5.0296, -1.2603,  ..., -1.9609, -1.5886, -2.0468],
        [ 1.2777, -2.4891, -0.9997,  ..., 13.5186, -5.0649, -2.2336],
        [-1.3080, -3.4765, -0.4379,  ..., 10.5144, -4.2653, -2.9360],
        ...,
        [ 0.8851, -0.0335, -4.2882,  ..., -1.6610,  1.6146, -0.4298],
        [-2.6111, -3.3815, -0.2003,  ...,  1.0606, -0.4910, -4.2268],
        [-2.6738, -3.0394, -0.2119,  ...,  0.2142, -0.4509, -3.7075]],
       grad_fn=<AddBackward0>)